## Import functions and define variables later used in loop

In [ ]:

import os
from google.cloud import bigquery
from google.oauth2 import service_account

my_files=os.listdir('C:\\Users\\brenn\\Documents\\ADA\\Wedge overall\\ada-wedge\\temp_clean')
pathtofiles = 'C:\\Users\\brenn\\Documents\\ADA\\Wedge overall\\ada-wedge\\temp_clean\\'


## Links to GBQ

In [ ]:
# links to client
service_path = "C:\\Users\\brenn\\Documents\\ADA\\Wedge overall\\"
service_file = 'ada-Wedge-b7fc47715ad0.json' # this is your authentication information  
gbq_proj_id = 'ada-wedge'  # change this to your project_id
gbq_dataset_id = 'wedge2' # and change this to your data set ID 

#####RENAME IT FROM TEST4 to wedge

private_key =service_path + service_file

# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## Creates a function to see if a table is in GBQ

In [ ]:
#looks for table using the client and table_ref
def tbl_exists(client, table_ref):
    from google.cloud.exceptions import NotFound
    try:
        client.get_table(table_ref)
        return True
    except NotFound:
        return False

## Creates the configuration for the table

In [ ]:

job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
job_config.schema_update_options = [
    bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION # This allows us to modify the table. 
]



job_config.schema = [
    bigquery.SchemaField("datetime", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("register_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("emp_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("upc", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("description", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_subtype", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("trans_status", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("department", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("quantity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Scale", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cost", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("unitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("total", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("regPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("altPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tax", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("taxexempt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("foodstamp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wicable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discountable", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("discounttype", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("voided", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("percentDiscount", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("ItemQtty", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volDiscType", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("volume", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("VolSpecial", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("mixMatch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("matched", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("memType", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("staff", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("numflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("itemstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("tenderstatus", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("charflag", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("varflag", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("batchHeaderID", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("local", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("organic", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("display", "BOOLEAN", mode="NULLABLE"),
    bigquery.SchemaField("receipt", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("card_no", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("store", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("branch", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("match_id", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("trans_id", "FLOAT", mode="NULLABLE"),
]
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1

## Creates table (if needed) and loads cleaned files into GBQ 

In [ ]:
path_to_files = "C:\\Users\\brenn\\Documents\\ADA\\Wedge overall\\ada-wedge\\temp_clean\\"

cleaned_files = os.listdir(path_to_files)

In [ ]:
##if a table doesn't exist, uses file name to name the table for each file in the table
for file in cleaned_files:
    my_table, junk = file.split(".") #splits on the period, anything after goes to junk, anything before goes to the variable my_table
    table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])
    
    
    if not tbl_exists(client, table_full_name) :
        table_ref = client.create_table(
            table = table_full_name
        )
    else :
        table_ref = client.get_table(table_full_name)
    table = client.get_table(table_ref)
    #print("Table {} contains {} columns".format(table_ref.table_id,len(table.schema)))

    
#opens the file in the folder, completes the job(loading the file) under the correct table name   
    with open(pathtofiles+file, "rb") as source_file:
        #print(file)
        job = client.load_table_from_file(
            source_file,
            table_ref,
            location="US",  # Must match the destination dataset location.
            job_config=job_config
        )  # API
        #break
    job.result()  # Waits for table load to complete and will give a result (gives an error if the table didnt load correctly)
    print(
        "Loaded {} rows into {}:{}.".format(
            job.output_rows, 'wedge2', table_ref.table_id
        )
    )


    table = client.get_table(table)
    #print("Table {} now contains {} columns.".format(table_ref.table_id, len(table.schema)))
    
#print('Finished all tables')